In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Metabolic_Rate"
cohort = "GSE101492"

# Input paths
in_trait_dir = "../../input/GEO/Metabolic_Rate"
in_cohort_dir = "../../input/GEO/Metabolic_Rate/GSE101492"

# Output paths
out_data_file = "../../output/preprocess/Metabolic_Rate/GSE101492.csv"
out_gene_data_file = "../../output/preprocess/Metabolic_Rate/gene_data/GSE101492.csv"
out_clinical_data_file = "../../output/preprocess/Metabolic_Rate/clinical_data/GSE101492.csv"
json_path = "../../output/preprocess/Metabolic_Rate/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE101492_family.soft.gz', 'GSE101492_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Metabolic_Rate/GSE101492/GSE101492_family.soft.gz
Matrix file: ../../input/GEO/Metabolic_Rate/GSE101492/GSE101492_series_matrix.txt.gz


Background Information:
!Series_title	"LONG NON-CODING RNAs ASSOCIATED WITH METABOLIC TRAITS IN HUMAN WHITE ADIPOSE TISSUE"
!Series_summary	"The aim of this study was to identify novel long noncoding RNAs (lncRNAs) that are differentially expressed in the subcutaneous region either in obesity or insulin resistance."
!Series_summary	"A number of novel lncRNAs were differentially expressed in the subcutaneous region either in obesity or insulin resistance. Two lncRNAs (termed here as adipocyte specific insulin resistance related lncRNAs (ADLNRIs), ADLNRI 1 and ADLNRI 2) were influenced by both conditions and enriched in adipocytes. Knockdown of either by antisense oligonucleotides in fat cellsadipocytes altered lipolysis and adiponectin secretion, respectively."
!Series_overall_design	"The 80 women included in this study were selected from the extremes of insulin sensitivity as measured by homeostasis model assessment of IR (HOMAIR) among 220 obese women who participated in a clinical tr

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import numpy as np
from typing import Optional, Callable, Dict, Any
import json
import gzip

# 1. Gene Expression Data Availability
# Based on the series title and summary, this is a study on lncRNAs in white adipose tissue
# lncRNAs are gene expression data, not miRNA or methylation data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the Sample Characteristics Dictionary:
# Key 3 shows 'insulin sensitivity: insulin sensitive' or 'insulin resistant' - this relates to metabolic rate
# Key 2 shows age values
# Key 1 shows gender, but it's constant (all female)

trait_row = 3  # insulin sensitivity is related to metabolic rate
age_row = 2    # age data is available
gender_row = None  # all subjects are female, so this is a constant feature

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> int:
    """Convert insulin sensitivity to binary values for metabolic rate.
    
    Insulin resistant (higher HOMAIR) = 1 (indicates impaired metabolic rate)
    Insulin sensitive (lower HOMAIR) = 0 (indicates normal metabolic rate)
    """
    if not isinstance(value, str):
        return None
    
    value = value.lower()
    if "insulin resistant" in value:
        return 1
    elif "insulin sensitive" in value:
        return 0
    else:
        return None

def convert_age(value: str) -> int:
    """Convert age string to integer value."""
    if not isinstance(value, str):
        return None
    
    if "age:" in value:
        try:
            return int(value.split("age:")[1].strip())
        except (ValueError, IndexError):
            return None
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender to binary (0 for female, 1 for male)."""
    # Not used in this dataset as all subjects are female
    if not isinstance(value, str):
        return None
    
    value = value.lower()
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load the clinical data from the matrix file properly
    # Read the matrix file using pandas with proper handling of gzipped file
    clinical_data = pd.read_csv(
        os.path.join(in_cohort_dir, "GSE101492_series_matrix.txt.gz"), 
        compression='gzip', 
        sep='\t', 
        comment='!', 
        header=None
    )
    
    # Extract clinical features using the provided function
    clinical_features_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age
        # gender_row and convert_gender are not provided since all subjects are female
    )
    
    # Preview extracted clinical features
    preview = preview_df(clinical_features_df)
    print("Preview of clinical features:", preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save extracted clinical features to CSV
    clinical_features_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of clinical features: {0: [nan, nan], 1: [nan, nan], 2: [nan, nan], 3: [nan, nan], 4: [nan, nan], 5: [nan, nan], 6: [nan, nan], 7: [nan, nan], 8: [nan, nan], 9: [nan, nan], 10: [nan, nan], 11: [nan, nan], 12: [nan, nan], 13: [nan, nan], 14: [nan, nan], 15: [nan, nan], 16: [nan, nan], 17: [nan, nan], 18: [nan, nan], 19: [nan, nan], 20: [nan, nan], 21: [nan, nan], 22: [nan, nan], 23: [nan, nan], 24: [nan, nan], 25: [nan, nan], 26: [nan, nan], 27: [nan, nan], 28: [nan, nan], 29: [nan, nan], 30: [nan, nan], 31: [nan, nan], 32: [nan, nan], 33: [nan, nan], 34: [nan, nan], 35: [nan, nan], 36: [nan, nan], 37: [nan, nan], 38: [nan, nan], 39: [nan, nan], 40: [nan, nan], 41: [nan, nan], 42: [nan, nan], 43: [nan, nan], 44: [nan, nan], 45: [nan, nan], 46: [nan, nan], 47: [nan, nan], 48: [nan, nan], 49: [nan, nan], 50: [nan, nan], 51: [nan, nan], 52: [nan, nan], 53: [nan, nan], 54: [nan, nan], 55: [nan, nan], 56: [nan, nan], 57: [nan, nan], 58: [nan, nan], 59: [nan, nan], 60: [nan, nan], 61:

/tmp/ipykernel_70236/1821478240.py:84: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  clinical_data = pd.read_csv(


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 70523
First 20 gene/probe identifiers:
Index(['18670005', '18670007', '18670009', '18670011', '18670020', '18670022',
       '18670023', '18670027', '18670028', '18670032', '18670033', '18670036',
       '18670038', '18670039', '18670041', '18670049', '18670051', '18670053',
       '18670054', '18670056'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analyzing the gene identifiers
# The identifiers like '18670005' appear to be probe IDs or array-specific identifiers 
# rather than standard human gene symbols (which would look like 'BRCA1', 'TP53', etc.)
# These numeric identifiers need to be mapped to standard gene symbols for meaningful analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['18670005', '18670007', '18670009', '18670011', '18670020'], 'probeset_id': ['18670005', '18670007', '18670009', '18670011', '18670020'], 'seqname': ['---', '---', '---', '---', '---'], 'strand': ['---', '---', '---', '---', '---'], 'start': ['---', '---', '---', '---', '---'], 'stop': ['---', '---', '---', '---', '---'], 'total_probes': [3.0, 1.0, 2.0, 2.0, 2.0], 'gene_assignment': ['--- // --- // Positive Housekeeping Controls - Probeset Type: normgene->exon // --- // ---', '--- // --- // Positive Housekeeping Controls - Probeset Type: normgene->exon // --- // ---', '--- // --- // Positive Housekeeping Controls - Probeset Type: normgene->exon // --- // ---', '--- // --- // Positive Housekeeping Controls - Probeset Type: normgene->exon // --- // ---', '--- // --- // Positive Housekeeping Controls - Probeset Type: normgene->exon // --- // ---'], 'mrna_assignment': ['---', '---', '---', '---', '---'], 'swissprot': ['---', '---', '---', '---', '---'], 'un

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Examine the gene annotation dataframe more comprehensively to find gene identifiers
print("Column names in gene annotation dataframe:")
print(gene_annotation.columns.tolist())

# Let's look at more columns that might contain gene information
sample_rows = gene_annotation.head(5)
print("\nSample rows with all columns from gene annotation dataframe:")
print(sample_rows)

# Looking closer at potential gene-containing columns
potential_gene_columns = ['gene_assignment', 'mrna_assignment', 'swissprot', 'unigene']
for col in potential_gene_columns:
    if col in gene_annotation.columns:
        print(f"\nSampling {col} column (first 5 non-null values):")
        sample_values = gene_annotation[col].dropna().head(5).tolist()
        for val in sample_values:
            print(val)

# Let's look at a different part of the dataframe
print("\nSampling rows in the middle of the dataframe:")
mid_index = len(gene_annotation) // 2
mid_sample = gene_annotation.iloc[mid_index:mid_index+5]
print(mid_sample[['ID', 'gene_assignment']])

# Based on our examination, we need to extract gene information differently
# We'll try a more comprehensive approach to identify gene symbols
def extract_genes_from_multiple_sources(row):
    """Extract gene symbols from various columns in the annotation"""
    # Initialize with an empty list
    genes = []
    
    # Try gene_assignment first
    if isinstance(row.get('gene_assignment'), str):
        assignment = row['gene_assignment']
        if assignment != '---' and 'Housekeeping Controls' not in assignment:
            parts = assignment.split('//')
            if len(parts) > 0 and parts[0].strip() != '---':
                genes.append(parts[0].strip())
    
    # Try other columns if available
    if isinstance(row.get('mrna_assignment'), str) and row['mrna_assignment'] != '---':
        mRNA_id = row['mrna_assignment'].split('//')[0].strip()
        if mRNA_id and mRNA_id != '---':
            genes.append(mRNA_id)
            
    # Use the ID itself if it appears to be a gene identifier
    if isinstance(row.get('ID'), str) and (row['ID'].startswith('NM_') or row['ID'].startswith('NR_')):
        genes.append(row['ID'])
    
    # Combine all found genes
    return genes if genes else None

# Apply our custom function to extract gene symbols
gene_annotation['Gene'] = gene_annotation.apply(extract_genes_from_multiple_sources, axis=1)

# Create gene mapping dataframe
valid_mapping = gene_annotation[['ID', 'Gene']].dropna()
print(f"\nNumber of rows with potential gene mapping: {len(valid_mapping)}")

# Since 'Gene' might contain lists, we need to explode it
mapping_exploded = valid_mapping.explode('Gene')
print(f"Number of rows after exploding Gene column: {len(mapping_exploded)}")
print("\nPreview of gene mapping dataframe:")
print(preview_df(mapping_exploded))

# 2. Create final gene mapping dataframe
gene_mapping = mapping_exploded[['ID', 'Gene']].copy()
print(f"\nFinal number of mappable probes: {len(gene_mapping)}")

# 3. Apply the gene mapping to convert probe-level data to gene expression data
print("\nApplying gene mapping...")
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Number of genes after mapping: {len(gene_data)}")
if len(gene_data) > 0:
    print("First 5 genes:")
    print(gene_data.index[:5])
else:
    print("Warning: No genes were mapped successfully.")

# Normalize gene symbols to ensure consistency
print("\nNormalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Number of genes after normalization: {len(gene_data)}")
if len(gene_data) > 0:
    print("First 5 normalized genes:")
    print(gene_data.index[:5])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to: {out_gene_data_file}")


Column names in gene annotation dataframe:
['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'gene_assignment', 'mrna_assignment', 'swissprot', 'unigene', 'category', 'locus type', 'notes', 'SPOT_ID']

Sample rows with all columns from gene annotation dataframe:
         ID probeset_id seqname strand start stop  total_probes  \
0  18670005    18670005     ---    ---   ---  ---           3.0   
1  18670007    18670007     ---    ---   ---  ---           1.0   
2  18670009    18670009     ---    ---   ---  ---           2.0   
3  18670011    18670011     ---    ---   ---  ---           2.0   
4  18670020    18670020     ---    ---   ---  ---           2.0   

                                     gene_assignment mrna_assignment  \
0  --- // --- // Positive Housekeeping Controls -...             ---   
1  --- // --- // Positive Housekeeping Controls -...             ---   
2  --- // --- // Positive Housekeeping Controls -...             ---   
3  --- // --- // Pos

---
---
---
---
---

Sampling unigene column (first 5 non-null values):


---
---
---
---
---

Sampling rows in the middle of the dataframe:
                      ID gene_assignment
2856336  TC0X000949.hg.1             NaN
2856337  TC0X000950.hg.1             NaN
2856338  TC0X000951.hg.1             NaN
2856339  TC0X000952.hg.1             NaN
2856340  TC0X000953.hg.1             NaN



Number of rows with potential gene mapping: 67362
Number of rows after exploding Gene column: 100032

Preview of gene mapping dataframe:
{'ID': ['TC01000001.hg.1', 'TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1'], 'Gene': ['NR_046018', 'NR_046018', 'ENST00000408384', 'ENST00000408384', 'NM_001005484']}

Final number of mappable probes: 100032

Applying gene mapping...


Number of genes after mapping: 182
First 5 genes:
Index(['BC000988', 'BC001023', 'BC001499', 'BC001846', 'BC001959'], dtype='object', name='Gene')

Normalizing gene symbols...
Number of genes after normalization: 0

Gene expression data saved to: ../../output/preprocess/Metabolic_Rate/gene_data/GSE101492.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {gene_data_normalized.shape}")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Re-examine the clinical data from the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Print out the sample characteristics to verify available rows
characteristics_dict = get_unique_values_by_row(clinical_data)
print("Sample characteristics dictionary:")
print(characteristics_dict)

# Define conversion functions for the clinical features based on the actual data
def convert_trait(value):
    """Convert treatment group to binary based on lycopene level (low=0, high=1)."""
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if 'high lycopene' in value:
        return 1.0  # High lycopene
    elif 'low lycopene' in value:
        return 0.0  # Low lycopene
    else:
        return None

def convert_age(value):
    """Convert age data to continuous values."""
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary (female=0, male=1)."""
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if 'male' in value:
        return 1.0
    elif 'female' in value:
        return 0.0
    else:
        return None

# Create the clinical dataframe using the correct row indices based on sample characteristics
try:
    # Row 3 contains treatment group (high/low lycopene) - this is our trait of interest
    # Row 2 contains age information
    # Row 1 contains gender information
    clinical_df = geo_select_clinical_features(
        clinical_data,
        trait="Metabolic_Rate",  # Using this as the trait name as per variable definition
        trait_row=3,  # Treatment group as the trait (based on lycopene levels)
        convert_trait=convert_trait,
        gender_row=1,  # Gender information is available in row 1
        convert_gender=convert_gender,
        age_row=2,     # Age information is available in row 2
        convert_age=convert_age
    )
    
    print("Clinical data preview:")
    print(preview_df(clinical_df.T))  # Transpose for better viewing
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Saved clinical data to {out_clinical_data_file}")
    
    # 3. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data_normalized)
    print(f"Shape of linked data: {linked_data.shape}")
    
    # 4. Handle missing values in the linked data
    linked_data_cleaned = handle_missing_values(linked_data, 'Metabolic_Rate')
    print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")
    
    # 5. Check if the trait and demographic features are biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, 'Metabolic_Rate')
    
    # 6. Validate the dataset and save cohort information
    note = "Dataset contains gene expression data from a study examining the effects of carotenoid-rich vegetables on metabolic syndrome in obese Japanese men. The trait variable is treatment group based on lycopene levels (0=Low lycopene, 1=High lycopene)."
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_trait_biased,
        df=unbiased_linked_data,
        note=note
    )
    
    # 7. Save the linked data if it's usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Saved processed linked data to {out_data_file}")
    else:
        print("Dataset validation failed. Final linked data not saved.")
        
except Exception as e:
    print(f"Error in processing clinical data: {e}")
    # If we failed to extract clinical data, update the cohort info
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=None,
        df=pd.DataFrame(),
        note="Failed to extract clinical data. Gene expression data is available but missing trait information."
    )
    print("Dataset validation failed due to missing clinical data. Only gene data saved.")

Shape of gene data after normalization: (0, 80)
Saved normalized gene data to ../../output/preprocess/Metabolic_Rate/gene_data/GSE101492.csv


Sample characteristics dictionary:
{0: ['suject status: obese'], 1: ['gender: female'], 2: ['age: 39', 'age: 28', 'age: 42', 'age: 30', 'age: 37', 'age: 36', 'age: 33', 'age: 27', 'age: 43', 'age: 44', 'age: 25', 'age: 35', 'age: 40', 'age: 29', 'age: 41', 'age: 34', 'age: 31', 'age: 38', 'age: 26', 'age: 32', 'age: 45'], 3: ['insulin sensitivity: insulin sensitive', 'insulin sensitivity: insulin resistant'], 4: ['tissue: subcutaneous white adipose tissue']}
Clinical data preview:
{'Metabolic_Rate': [nan, nan, nan, nan, nan], 'Age': [39.0, 28.0, 42.0, 30.0, 42.0], 'Gender': [1.0, 1.0, 1.0, 1.0, 1.0]}
Saved clinical data to ../../output/preprocess/Metabolic_Rate/clinical_data/GSE101492.csv
Shape of linked data: (80, 3)
Shape of linked data after handling missing values: (0, 2)
Quartiles for 'Metabolic_Rate':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Metabolic_Rate' in this dataset is fine.

Quartiles for 'Age':
  25%: nan
  50% (Median)